# 🎵 Machine Listening Workshop - Demo

Welcome to the **Machine Listening** workshop! This notebook demonstrates audio processing and analysis using Python.

## 📦 Environment Setup

First, let's set up the environment automatically.

In [ ]:
# Detects if running on Colab and installs dependencies
import sys
import subprocess

try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running on Google Colab")
    
    # Repository URL
    REPO_URL = "https://raw.githubusercontent.com/zepadovani/2025_FU_workshop/main"
    REQUIREMENTS_URL = f"{REPO_URL}/requirements.txt"
    
    print("\n📦 Installing dependencies...")
    
    # Tries to download and install from requirements.txt
    try:
        subprocess.run(["wget", "-q", REQUIREMENTS_URL, "-O", "requirements.txt"], check=True)
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"], check=True)
        print("✅ Dependencies installed from requirements.txt")
    except:
        # Fallback: manual installation
        print("⚠️  Installing packages manually...")
        packages = ["librosa", "soundfile", "gradio", "matplotlib"]
        for pkg in packages:
            subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=True)
            print(f"  ✓ {pkg}")
        print("✅ Installation complete")
    
except ImportError:
    IN_COLAB = False
    print("💻 Running locally (using Pixi environment)")

## 📚 Import Libraries

Now let's import the necessary libraries.

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import gradio as gr
from IPython.display import Audio, display

print("✅ Libraries imported successfully!")
print(f"Librosa version: {librosa.__version__}")
print(f"Gradio version: {gr.__version__}")

## 🎼 Generate Synthetic Audio

Let's create an example audio for demonstration.

In [ ]:
# Parameters
sample_rate = 22050
duration = 2.0  # seconds
frequency = 440  # Hz (A note)

# Generates a sine tone
t = np.linspace(0, duration, int(sample_rate * duration))
audio_signal = 0.5 * np.sin(2 * np.pi * frequency * t)

# Adds a fade in/out
fade_samples = int(0.1 * sample_rate)
audio_signal[:fade_samples] *= np.linspace(0, 1, fade_samples)
audio_signal[-fade_samples:] *= np.linspace(1, 0, fade_samples)

print(f"✅ Audio generated: {duration}s @ {sample_rate} Hz")
print(f"   Frequency: {frequency} Hz")
print(f"   Shape: {audio_signal.shape}")

# Plays the audio
display(Audio(audio_signal, rate=sample_rate))

## 📊 Spectral Analysis

Let's visualize the audio spectrogram.

In [ ]:
# Calculates the STFT (Short-Time Fourier Transform)
D = librosa.stft(audio_signal)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

# Visualizes the spectrogram
plt.figure(figsize=(12, 4))
librosa.display.specshow(S_db, x_axis='time', y_axis='hz', sr=sample_rate, cmap='viridis')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.tight_layout()
plt.show()

print(f"Spectrogram shape: {S_db.shape}")

## 🎯 Feature Analysis

Let's extract audio features using Librosa.

In [ ]:
# Duration
duration_calc = librosa.get_duration(y=audio_signal, sr=sample_rate)

# Tempo (BPM) - for a pure tone, might not be very meaningful
tempo, beats = librosa.beat.beat_track(y=audio_signal, sr=sample_rate)

# Zero Crossing Rate
zcr = librosa.feature.zero_crossing_rate(audio_signal)

# Spectral Centroid
spectral_centroids = librosa.feature.spectral_centroid(y=audio_signal, sr=sample_rate)

# MFCCs
mfccs = librosa.feature.mfcc(y=audio_signal, sr=sample_rate, n_mfcc=13)

print("📊 Extracted Features:")
print(f"  Duration: {duration_calc:.2f}s")
print(f"  Tempo: {tempo:.2f} BPM")
print(f"  Zero Crossing Rate (mean): {zcr.mean():.4f}")
print(f"  Spectral Centroid (mean): {spectral_centroids.mean():.2f} Hz")
print(f"  MFCCs shape: {mfccs.shape}")

## 🎨 Interactive Gradio Interface

Let's create an interactive web interface with Gradio!

In [ ]:
def analyze_audio(audio_input):
    """
    Analyzes an audio file and returns information + spectrogram.
    """
    try:
        # Process input
        if isinstance(audio_input, tuple):
            sr, y = audio_input
            # Convert to mono if necessary
            if len(y.shape) > 1:
                y = np.mean(y, axis=1)
            # Normalize
            y = y.astype(np.float32)
            if y.max() > 1.0:
                y = y / np.iinfo(np.int16).max
        else:
            y, sr = librosa.load(audio_input, sr=None)
        
        # Analysis
        duration = librosa.get_duration(y=y, sr=sr)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y).mean()
        centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
        
        # Spectrogram
        D = librosa.stft(y)
        S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
        
        fig, ax = plt.subplots(figsize=(10, 4))
        librosa.display.specshow(S_db, x_axis='time', y_axis='hz', sr=sr, ax=ax, cmap='viridis')
        fig.colorbar(ax.collections[0], ax=ax, format='%+2.0f dB')
        ax.set_title('Spectrogram')
        plt.tight_layout()
        
        # Textual result
        result = f"""
### ✅ Analysis Complete!

**📊 Information:**
- Duration: {duration:.2f}s
- Sample Rate: {sr} Hz
- Estimated Tempo: {tempo:.2f} BPM
- Zero Crossing Rate: {zcr:.4f}
- Spectral Centroid: {centroid:.2f} Hz
        """
        
        return result, fig
        
    except Exception as e:
        return f"❌ Error: {str(e)}", None


# Create interface
with gr.Blocks(title="Machine Listening Demo") as demo:
    gr.Markdown("""
    # 🎵 Machine Listening - Interactive Demo
    Upload an audio file or record using your microphone!
    """)
    
    with gr.Row():
        audio_input = gr.Audio(label="Audio", type="filepath", sources=["upload", "microphone"])
    
    analyze_btn = gr.Button("🔍 Analyze", variant="primary")
    
    text_output = gr.Markdown()
    plot_output = gr.Plot()
    
    analyze_btn.click(
        fn=analyze_audio,
        inputs=audio_input,
        outputs=[text_output, plot_output]
    )

# Launch interface (share=True for public link on Colab)
demo.launch(share=IN_COLAB)